In [1]:
import pandas as pd

In [2]:
pd.options.display.max_columns = None  # Remove "dots" from display when printing dataframes

# Read data

In [3]:
df_student = pd.read_excel('data/test.xlsx', sheet_name='estudiante')
df_list = pd.read_excel('data/test.xlsx', sheet_name='listado')
df_professor = pd.read_excel('data/test.xlsx', sheet_name='profesor')
df_group = pd.read_excel('data/test.xlsx', sheet_name='grupo')
df_test = pd.read_excel('data/test.xlsx', sheet_name='examen')
df_appoint = pd.read_excel('data/test.xlsx', sheet_name='cita')

# Preprocesing

In [4]:
df_test = df_test.fillna(0)

In [5]:
# Format string to convert to datetime
replacement_values = {
    'a.m.':'AM', 
    'p.m.':'PM'
    }

for old_value, new_value in replacement_values.items():
    df_appoint['horaExamen'] = df_appoint['horaExamen'].str.replace(old_value, new_value, regex=True)

In [6]:
df_test['inicio'] = pd.to_datetime(df_test['inicio'], format='%H:%M:%S')
df_test['final'] = pd.to_datetime(df_test['final'], format='%H:%M:%S')
df_appoint['horaExamen'] = pd.to_datetime(df_appoint['horaExamen'], format='%H:%M %p')

# Questions

## Q1
What is the ranking of teachers according to the average grade obtained by their students?

In [7]:
df_ProGro = pd.merge(df_professor, df_group, how='inner', on='IDP')
df_StuLis = pd.merge(df_student, df_list, how='inner', on='IDE')
df1 = pd.merge(df_ProGro, df_StuLis, on='IDG')

df1 = df1.groupby('profesor').aggregate({'prom':'mean'}).reset_index()
df1.sort_values(by='prom', inplace=True, ascending=False)

df1.head()

,profesor,prom
18,Odiseo,3.902778
13,Hefesto,3.784884
21,Polidoro,3.781250
14,Hermes,3.635417
11,Euforbo,3.565217


## Q2
Which were the students who passed the test with more or equal to 4.0, who is their professor, at what time of the day do they have classes and to which group do they belong?

In [8]:
df2 = pd.merge(df_ProGro, df_StuLis, on='IDG')
df2 = pd.merge(df2, df_test, on='IDE')

df2 = df2.query("nota >= 4.0")

df2[['IDE', 'nota', 'profesor', 'jornada', 'IDG']].head()

,IDE,nota,profesor,jornada,IDG
2,1585,4.0,Patroclo,día,G11
4,1643,5.0,Patroclo,día,G11
6,1800,4.5,Patroclo,día,G11
9,1938,4.0,Patroclo,día,G11
15,1975,4.5,Patroclo,día,G11


## Q3
What were the exam averages per hour of study and per academic program (sorted by average)?

In [9]:
df3 = pd.merge(df_ProGro, df_StuLis, on='IDG')

df3 = df3.groupby(['programa', 'jornada'])['prom'].mean().reset_index()
df3.sort_values(by='prom', ascending=False, inplace=True)

df3.head()

,programa,jornada,prom
7,Ingeniería Industrial,noche,3.493056
4,Ingeniería Electrónica,día,3.187500
5,Ingeniería Electrónica,noche,3.151515
6,Ingeniería Industrial,día,3.125000
8,Ingeniería Mecánica,día,3.083333


## Q4
You want to compare the students who took the test with respect to the time they took. What is the difference in averages between the scores of the 40 fastest students versus the averages of the 40 slowest students?

In [10]:
df4 = df_test.copy()

df4['duracion'] = df4['final'] - df4['inicio']

dfFas = df4[:40]
dfSlo = df4[:-40]

mean_faster = dfFas['nota'].mean()
mean_slower = dfSlo['nota'].mean()

print(f'Average grade of the fastest students: {round(mean_faster, 2)}\nAverage grade of the slowest students: {round(mean_slower, 2)}')

Average grade of the fastest students: 2.02
Average grade of the slowest students: 1.9


## Q5
Taking into account the names of the teachers, how many students passed (grade 3 or higher) the test and how many failed?

In [11]:
df5 = pd.merge(df_ProGro, df_StuLis, on='IDG')
df5 = pd.merge(df5, df_test, on='IDE')

df5['aprobation'] = df5['nota'].apply(lambda x: 'yes' if x >= 3 else 'no')
df5 = df5.groupby(['profesor', 'aprobation'])['IDE'].count().reset_index()

df5.head(10)

,profesor,aprobation,IDE
0,Agamenón,no,31
1,Agamenón,yes,8
2,Agenor,no,13
3,Agenor,yes,8
4,Andrómaca,no,10
5,Andrómaca,yes,3
6,Apolo,no,9
7,Apolo,yes,13
8,Aquiles,no,13
9,Aquiles,yes,4


## Q6
The test questions should be arranged from the most difficult to the easiest according to the students' answers.

In [12]:
df6 = df_test[[
    'p1', 'p2', 'p3', 'p4', 'p5', 
    'p6', 'p7', 'p8', 'p9', 'p10'
    ]].copy().sum()

df6.sort_values(ascending=True)

p6      72.0
p7      84.5
p2      85.0
p3      86.0
p1      93.0
p5     102.5
p4     111.0
p8     117.0
p9     120.5
p10    121.0
dtype: float64

## Q7
Considering the relationship between academic performance and circadian rhythms presented in a [recent study](https://academic.oup.com/ej/article-abstract/133/652/1407/6888010?redirectedFrom=fulltext&login=false). Will there be a difference between the grade point average of students who took the test in the daytime versus the grade point average of students who took the test in the evening?

In [13]:
df7 = pd.merge(df_ProGro, df_StuLis, on='IDG')
df7 = pd.merge(df7, df_test, on='IDE')

df7_day = df7.query("jornada == 'día'")
df7_night = df7.query("jornada == 'noche'")

mean_day = df7_day['nota'].mean()
mean_night = df7_night['nota'].mean()

print(f'The average grade for daytime students is {round(mean_day, 2)} compared to {round(mean_night, 2)} for nighttime students.')

The average grade for daytime students is 1.99 compared to 1.78 for nighttime students.


## Q8
What is the top 10 professors with the highest percentage of hours in terms of non-attendance by their students?

In [14]:
df8 = pd.merge(df_ProGro, df_StuLis, on='IDG')
df8 = pd.merge(df8, df_test, on='IDE')

df8 = df8.groupby('profesor').aggregate({'fallaP':'sum'}).reset_index()
df8.sort_values(by='fallaP', ascending=False, inplace=True)

df8.head(10)

,profesor,fallaP
16,Menelao,517.61
8,Diomedes,335.30
23,Príamo,329.41
5,Artemisa,188.22
9,Dolón,152.92
22,Poseidón,129.40
2,Andrómaca,129.39
20,Patroclo,82.34
6,Atenea,58.81
17,Néstor,41.17


## Q9
Which of the students who took the test are not listed?

In [15]:
df9 = pd.merge(df_test[['IDE', 'nota']], 
               df_list[['IDE', 'IDG', 'prom', 'fallas']], 
               how='inner', on='IDE'
               )

students = df9['IDE'].tolist()

print(f'Unlisted students:')
for student in students:
    print(student)

Unlisted students:
1502
1503
1504
1505
1507
1508
1509
1510
1511
1512
1513
1514
1515
1516
1518
1519
1520
1521
1522
1525
1526
1529
1530
1531
1532
1533
1534
1535
1536
1537
1538
1539
1540
1541
1542
1543
1544
1545
1546
1547
1548
1549
1550
1551
1552
1553
1554
1555
1557
1559
1560
1561
1562
1563
1564
1565
1566
1567
1568
1569
1570
1571
1572
1573
1575
1576
1577
1578
1579
1580
1582
1583
1584
1585
1587
1589
1592
1593
1594
1595
1596
1598
1600
1601
1602
1603
1604
1607
1608
1609
1610
1611
1612
1613
1614
1615
1616
1617
1618
1620
1621
1622
1623
1625
1627
1628
1629
1631
1632
1634
1635
1636
1637
1638
1639
1640
1641
1643
1644
1645
1646
1648
1649
1651
1652
1653
1654
1655
1656
1657
1658
1659
1660
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672
1673
1674
1675
1676
1677
1678
1680
1681
1681
1683
1684
1685
1686
1687
1688
1689
1690
1692
1696
1697
1698
1699
1700
1701
1702
1703
1704
1705
1706
1707
1708
1709
1710
1711
1712
1713
1714
1715
1716
1717
1719
1720
1723
1725
1726
1727
1728
1730
1731
1733
1734
1736
1